In [2]:
from tkinter import *
import pandas as pd
from path import Path
import hashlib
from datetime import datetime
import Place_Bet
import json
import pprint as pprint
import warnings
warnings.filterwarnings('ignore')
pptint = pprint.PrettyPrinter(indent=4)
from IPython.display import clear_output

In [4]:
class App:

    def __init__(self, master):

        frame = Frame(master)
        frame.pack()

        self.button = Button(
            frame, text="Log Out", fg="red", command=frame.quit
            )
        self.button.pack(side=LEFT)

        self.new_user = Button(frame, text="New User", command=self.new_user)
        self.new_user.pack(side=LEFT)
        
        self.returning_user = Button(frame, text="Returning User", command=self.returning_user)
        self.returning_user.pack(side=LEFT)
        
        
    def new_user(self):
        frame = Frame(master)
        frame.pack()
        with open('transaction_history.txt') as json_file:
            transaction_history = json.load(json_file)
        csv_path = Path('user_info_dataframe.csv')
        user_info_dataframe = pd.read_csv(csv_path)
        user_info_dataframe.drop(columns = ['Unnamed: 0'],inplace=True)
        username = input('Please input the username that you would like to use: ')
        for row in user_info_dataframe['Username']:
            if username == row:
                print('\nThe username that you have provided is already in use, please select another username.')
                return
        print('\nYour password must be at least 8 characters long and contain at least one capital letter, one lowercase letter, one number, and one special character.')
        password_count = 0
        while password_count < 4:
            password_count += 1
            password = input('\nPlease input the password that you would like to use: ')
            if len(password) < 8:
                print('Your password must be at least eight characters long. Please try again.')
                confirm_password = ''
            else:
                confirm_password = input('\nPlease reenter your password: ')
                confirm_password  = bytes(confirm_password, 'utf-8')
            password  = bytes(password, 'utf-8')
            if confirm_password != password:
                print('\nThe passwords that you entered do not match.')
            elif confirm_password == password:
                password = hashlib.sha256(password).hexdigest()
                password_count += 10
        if password_count == 4:
            print('You have exceeded the maximum number of attempts, please restart and try again.')
            return
        email_count = 0
        while email_count < 4:
            email_count += 1
            user_email = input('\nPlease provide us with the email that you would like to be associated with this account: ')
            for row in user_info_dataframe['Email_Address']:
                if user_email == row:
                    print('\nThe email address that you have provided is already in use, please use a different email address.')
                    return
            confirm_user_email = input('\nPlease reenter your email address: ')
            if confirm_user_email != user_email:
                print('\nThe email address that you entered does not match the original email that was provided. Please try again.')
            elif confirm_user_email == user_email:
                # Append email address to dataframe
                email_count += 10
        user_account_address = input('\nPlease enter your Kovan Testnet public account address. If you do not have a Kovan Testnet account, please get one and try again: ')
        member_since = datetime.now().strftime('%m-%d-%Y')
        user_info_dataframe = user_info_dataframe.append({'BetCoins':100, 'Username':username, 'Password':password, 'Email_Address': user_email, 'User_Kovan_Testnet_Address': user_account_address, 'Member_Since': member_since} , ignore_index=True)
        user_info_dataframe.to_csv('user_info_dataframe.csv')
        transaction_history['Transactions'].append({
            'Username':username,
            'Transaction_Type': 'Account_Initialization',
            'Transaction_Amount': 100,
            'Transaction_Time': datetime.now().strftime('%m-%d-%y')
        })

        with open('transaction_history.txt','w') as outfile:
            json.dump(transaction_history, outfile)
    def account_actions(user_index,check_user_info,user_info_dataframe,username):
        frame = Frame(master)
        frame.pack()
        print('Please choose an action.')
        action = 'Initial'
        balance = int(check_user_info['BetCoins'][user_index])
        exit_list = ['Exit','exit']
        check_user_info = user_info_dataframe[user_info_dataframe['Username'] == username]
        csv_path = Path('user_info_dataframe.csv')
        user_info_dataframe = pd.read_csv(csv_path)
        user_info_dataframe.drop(columns = ['Unnamed: 0'],inplace=True)
        def balance(self):
            balance = int(check_user_info['BetCoins'][user_index])
            print(f"Your account balance is {balance} BetCoins.")
            print('You may also view your account balance in your Ethereum wallet.')

        def place_bet(self):
            new_balance = Place_Bet.place_bet(balance,check_user_info,user_index,user_info_dataframe)
            check_user_info['BetCoins'][user_index] = new_balance

        def transaction_history(self):
            with open('transaction_history.txt') as json_file:
                transaction_history = json.load(json_file)
            pprint.pprint(transaction_history)
            # Attach each transaction with the username, the date that the transaction occured, and the bet amount. When called, users can select a month or just grab every transaction
            # Print out nothing but the three items, no public keys or anything else, just date, user, and bet amount

        def leaderboard(self):
            leaderboard = user_info_dataframe[['Username','BetCoins']]
            new_index = range(1,len(leaderboard) + 1)
            leaderboard['Standing'] = new_index
            leaderboard = leaderboard.set_index('Standing')
            leaderboard_show = leaderboard[0:9]
            if user_index > 10:
                leaderboard_show.append(leaderboard[user_index + 1])
            print(leaderboard_show)
        frame = Frame(master)
        frame.pack()
    
        self.balance = Button(frame, text="Account Balance", command=self.balance)
        self.balance.pack(side=CENTER)

        self.place_bet = Button(frame, text="Place Bet", command=self.place_bet)
        self.place_bet.pack(side=CENTER)

        self.transaction_history = Button(frame, text="Transaction History", command=self.transaction_history)
        self.transaction_history.pack(side=CENTER)

        self.leaderboard = Button(frame, text="Leader Board", command=self.leaderboard)
        self.leaderboard.pack(side=CENTER)
        
        
        
    def returning_user(self):
        frame = Frame(master)
        frame.pack()
        csv_path = Path('user_info_dataframe.csv')
        user_info_dataframe = pd.read_csv(csv_path)
        user_info_dataframe.drop(columns = ['Unnamed: 0'],inplace=True)
        username = input('Please input the username that you would like to use: ')
        check_user_info = user_info_dataframe[user_info_dataframe['Username'] == username]
        if len(check_user_info) == 0:
            print('The username that you have entered does not exist. Please make sure that you have input the correct username or create a new account.')
            return
        password_count = 0
        user_index = check_user_info.index[0]
        while password_count < 4:
            password_count += 1
            password = input('\nPlease input your password: ')
            password  = bytes(password, 'utf-8')
            password = hashlib.sha256(password).hexdigest()
            user_password = check_user_info['Password'][user_index]
            if user_password == password:
                password_count += 10
            elif user_password != password:
                print('\nThe passwords that you entered is incorrect.')
        if password_count == 4:
            print('You have exceeded the maximum number of attempts, please restart and try again.')
            return
        print('Loading Account...')
        clear_output()
        self.balance = Button(frame, text="Account Balance", command=self.balance)
        self.balance.pack(side=CENTER)

        self.place_bet = Button(frame, text="Place Bet", command=self.place_bet)
        self.place_bet.pack(side=CENTER)

        self.transaction_history = Button(frame, text="Transaction History", command=self.transaction_history)
        self.transaction_history.pack(side=CENTER)

        self.leaderboard = Button(frame, text="Leader Board", command=self.leaderboard)
        self.leaderboard.pack(side=CENTER)
        print('Please choose an action.')
        action = 'Initial'
        balance = int(check_user_info['BetCoins'][user_index])
        exit_list = ['Exit','exit']
        check_user_info = user_info_dataframe[user_info_dataframe['Username'] == username]
        csv_path = Path('user_info_dataframe.csv')
        user_info_dataframe = pd.read_csv(csv_path)
        user_info_dataframe.drop(columns = ['Unnamed: 0'],inplace=True)
        def balance(self):
            balance = int(check_user_info['BetCoins'][user_index])
            print(f"Your account balance is {balance} BetCoins.")
            print('You may also view your account balance in your Ethereum wallet.')

        def place_bet(self):
            new_balance = Place_Bet.place_bet(balance,check_user_info,user_index,user_info_dataframe)
            check_user_info['BetCoins'][user_index] = new_balance

        def transaction_history(self):
            with open('transaction_history.txt') as json_file:
                transaction_history = json.load(json_file)
            pprint.pprint(transaction_history)
            # Attach each transaction with the username, the date that the transaction occured, and the bet amount. When called, users can select a month or just grab every transaction
            # Print out nothing but the three items, no public keys or anything else, just date, user, and bet amount

        def leaderboard(self):
            leaderboard = user_info_dataframe[['Username','BetCoins']]
            new_index = range(1,len(leaderboard) + 1)
            leaderboard['Standing'] = new_index
            leaderboard = leaderboard.set_index('Standing')
            leaderboard_show = leaderboard[0:9]
            if user_index > 10:
                leaderboard_show.append(leaderboard[user_index + 1])
            print(leaderboard_show)
    
    
        
    
        
        
    
root = Tk()

app = App(root)

root.mainloop()
root.destroy()

AttributeError: 'App' object has no attribute 'new_user'